### Authorship
@author: Alexandre Pereira Santos <br>
alexandre.santos(at)lmu.de<br>
- many scripts in this notebook com from geemap (see below)

### Tasks
- get Global Human Settlement Layer population estimates data using the Google Earth Engine functionalities
- clip it to an AOI

### Prerequisites
- earth engine api (ee) https://developers.google.com/earth-engine/apidocs
- gee map (gee), https://geemap.org/ by Qiusheng Wu
- create an account and a Google Cloud project in Google Developer, see here: https://developers.google.com/earth-engine/guides/auth

# init

In [ ]:
import ee
import geemap
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import logging
# extract coordinates from the bounding box
def get_coords(gdf):
    coords = gdf.to_crs(epsg='4326').envelope
    epsg_coords = coords.crs.to_epsg()
    transform = [coords.bounds.values[0][0], coords.bounds.values[0][1], coords.bounds.values[0][2], coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    rec_roi = ee.Geometry.Rectangle(transform[0],transform[1],transform[2],transform[3])
    coi = roi.centroid(maxError=1)
    return epsg_coords, roi, rec_roi, coi


In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='') # Replace with your GEE project
print(ee.__version__)

In [ ]:
# check if the authentication is working
print(ee.Image("JRC/GHSL/P2023A/GHS_POP/1975").getInfo())

# imports

In [ ]:
#input a vector and a raster file
AOI_path = Path('../data/processed/')

AOI_file_JAK = 'JAK_LIM_BoundingBox_AOI_A.shp' #Jakarta
AOI_file_MUM = 'MUM_LIM_BoundingBox_AOI_A.shp' #Mumbai
AOI_file_MAN = 'MAN_LIM_BoundingBox_AOI_A.shp' #Manila

#read the vector files
AOI_gdf_JAK = gpd.read_file(AOI_path/AOI_file_JAK).to_crs(epsg=4326)
AOI_gdf_MUM = gpd.read_file(AOI_path/AOI_file_MUM).to_crs(epsg=4326)
AOI_gdf_MAN = gpd.read_file(AOI_path/AOI_file_MAN).to_crs(epsg=4326)

# get the coordinates
JAK_aoi_crs, JAK_roi, JAK_rec_roi, JAK_coi = get_coords(AOI_gdf_JAK)
MUM_aoi_crs, MUM_roi, MUM_rec_roi, MUM_coi = get_coords(AOI_gdf_MUM)
MAN_aoi_crs, MAN_roi, MAN_rec_roi, MAN_coi = get_coords(AOI_gdf_MAN)

print(JAK_aoi_crs,MUM_aoi_crs,MAN_aoi_crs)

# get

In [ ]:
# methods from https://www.geo.fu-berlin.de/en/v/geo-it/gee/2-monitoring-ndvi-nbr/2-2-calculating-indices/ndvi-s2/index.html

def roi_ndvi(aoi_gdf):
    aoi_coords = aoi_gdf.to_crs(epsg='4326').envelope
    #print('N',aoi_coords.bounds.values[0][3],'S',aoi_coords.bounds.values[0][1],'W',aoi_coords.bounds.values[0][0],'E',aoi_coords.bounds.values[0][2])
    transform = [aoi_coords.bounds.values[0][0], aoi_coords.bounds.values[0][1], aoi_coords.bounds.values[0][2], aoi_coords.bounds.values[0][3]]
    roi = ee.Geometry.BBox(west=transform[0], south=transform[1], east=transform[2], north=transform[3])
    
    s2=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi).filterDate('2020-01-01', '2020-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 7)).median()
    # Use median to reduce noise and clouds

    # Calculate NDVI
    ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')  

    return ndvi

In [ ]:
JAK_ndvi = roi_ndvi(AOI_gdf_JAK)

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(JAK_roi, 10)
Map.addLayer(JAK_ndvi.clip(JAK_roi), ndviParams, 'NDVI')
Map

In [ ]:
MUM_ndvi = roi_ndvi(AOI_gdf_MUM)

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(MUM_roi, 10)
Map.addLayer(MUM_ndvi.clip(MUM_roi), ndviParams, 'NDVI')
Map

In [ ]:
MAN_ndvi = roi_ndvi(AOI_gdf_MAN)

# Define visualization parameters
ndviParams = {min: 0,max: 1,'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
# Visualize the NDVI
Map.centerObject(MAN_roi, 10)
Map.addLayer(MAN_ndvi.clip(MAN_roi), ndviParams, 'NDVI')
Map

In [ ]:
# define a function that exports the image to Google Drive
def export_image(image, filename, roi): #crs, projection, removed these arguments as they caused a resampling error
    try: 
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=filename,
            folder='gee_exports',
            region=roi,
            #crs=crs,
            #crsTransform=projection['transform'],
            maxPixels=1e13,
            fileFormat='GeoTIFF',
            formatOptions={
                'cloudOptimized': False
            }
        )
        task.start()
        logging.info(f'Exporting {filename} to Google Drive')
        return task
    except Exception as e:
        logging.error(f'An error has occurred: {e}')
        return None

# check image stats before export
def check_image_stats(image, roi): 
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=250,
        maxPixels=1e9       
    )
    return stats.getInfo()


In [ ]:
JAK_ndvi_clip = JAK_ndvi.clip(JAK_rec_roi)
MUM_ndvi_clip = MUM_ndvi.clip(MUM_rec_roi)
MAN_ndvi_clip = MAN_ndvi.clip(MAN_rec_roi)
print('JAK_NDVI stats:', check_image_stats(JAK_ndvi_clip, JAK_rec_roi))
print('MUM_NDVI stats:', check_image_stats(MUM_ndvi_clip, MUM_rec_roi))
print('MAN_NDVI stats:', check_image_stats(MAN_ndvi_clip, MAN_rec_roi))


In [ ]:
# define a function that exports the image to Google Drive
def gee_to_tiff(input_layer, output_filename, roi):
    out_folder = 'gee_exports'
    #projection = input_layer.projection().getInfo()
    
    try:
        task = geemap.ee_export_image_to_drive(
            input_layer,
            description=output_filename,
            folder=out_folder,
            maxPixels=1e13, #ressamples to a low resolution
            region = roi,
            #crs = projection['crs'],
            fileFormat = 'GeoTIFF',
            scale = 10           
        )
        #task.start()
        logging.info(f'Exporting {output_filename} to Google Drive')
        return task 
    except Exception as e:
        logging.error(f'An error has occurred: {e}')
        return None

# export

In [ ]:
input_layer = MUM_ndvi_clip
output_filename = 'MUM_ENV_NDVI_Sentinel2_2020'
roi = MUM_rec_roi
out_folder = 'gee_exports'

export = geemap.ee_export_image_to_drive(
            input_layer,
            description=output_filename,
            folder=out_folder,
            maxPixels=1e13, #ressamples to a low resolution
            region = roi,
            crs = 'EPSG:4326',
            fileFormat = 'GeoTIFF',
            scale = 10           
        )

In [ ]:
JAK_task = gee_to_tiff(JAK_ndvi_clip, 'JAK_ENV_NDVI_Sentinel2_2000', JAK_rec_roi) 
MUM_task = gee_to_tiff(MUM_ndvi_clip, 'MUM_ENV_NDVI_Sentinel2_2000', MUM_rec_roi) 
MAN_task = gee_to_tiff(MAN_ndvi_clip, 'MAN_ENV_NDVI_Sentinel2_2000', MAN_rec_roi) 

# check task status
tasks = [JAK_task, MUM_task, MAN_task]
for task in tasks: 
    if task: 
        print(f'Task {task.status()["id"]} is {task.status()["state"]}')
    else:
        print('Task failed')
